In [1]:
import numpy as np
import pandas as pd
import io
import re
import sys
!{sys.executable} -m pip install gensim
!{sys.executable} -m pip install nltk
import gensim
import nltk
nltk.download('punkt') #nltk Dependency
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize
from datetime import datetime,date,timedelta


    100% |████████████████████████████████| 23.6MB 1.8MB/s 
    100% |████████████████████████████████| 1.4MB 16.1MB/s 
    100% |████████████████████████████████| 133kB 28.2MB/s 
    100% |████████████████████████████████| 61kB 23.3MB/s 
    100% |████████████████████████████████| 5.1MB 7.3MB/s 
    100% |████████████████████████████████| 552kB 25.3MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
newsDataframe = pd.read_csv('drive/My Drive/stockNewsDataset/newsdata.csv')
marketDataframe = pd.read_csv('drive/My Drive/stockNewsDataset/marketdata.csv')

In [4]:
newsDataframe.head()

,Unnamed: 0,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,...,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,0,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,e58c6279551b85cf,China's Daqing pumps 43.41 mln tonnes of oil i...,3,1,RTRS,"{'ENR', 'ASIA', 'CN', 'NGS', 'EMRG', 'RTRS', '...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,7.0
1,1,2007-01-01 07:03:35+00:00,2007-01-01 07:03:34+00:00,2007-01-01 07:03:34+00:00,5a31c4327427f63f,"FEATURE-In kidnapping, finesse works best",3,1,RTRS,"{'FEA', 'CA', 'LATAM', 'MX', 'INS', 'ASIA', 'I...",...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0
2,2,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,1cefd27a40fabdfe,PRESS DIGEST - Wall Street Journal - Jan 1,3,1,RTRS,"{'RET', 'ENR', 'ID', 'BG', 'US', 'PRESS', 'IQ'...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,11.0,17.0
3,3,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,13.0,15.0
4,4,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
marketDataframe.head()

,Unnamed: 0,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe
0,0,2007-02-01 22:00:00+00:00,A.N,Agilent Technologies Inc,2606900.0,32.19,32.17,0.005938,0.005312,NaN,NaN,-0.001860,0.000622,NaN,NaN,0.034672,1.0
1,1,2007-02-01 22:00:00+00:00,AAI.N,AirTran Holdings Inc,2051600.0,11.12,11.08,0.004517,-0.007168,NaN,NaN,-0.078708,-0.088066,NaN,NaN,0.027803,0.0
2,2,2007-02-01 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1164800.0,37.51,37.99,-0.011594,0.025648,NaN,NaN,0.014332,0.045405,NaN,NaN,0.024433,1.0
3,3,2007-02-01 22:00:00+00:00,AAPL.O,Apple Inc,23747329.0,84.74,86.23,-0.011548,0.016324,NaN,NaN,-0.048613,-0.037182,NaN,NaN,-0.007425,1.0
4,4,2007-02-01 22:00:00+00:00,ABB.N,ABB Ltd,1208600.0,18.02,18.01,0.011791,0.025043,NaN,NaN,0.012929,0.020397,NaN,NaN,-0.017994,1.0


In [6]:
%%time
inputMktData = pd.DataFrame()
inputMktData['date'] = marketDataframe['time']
inputMktData['date'] =  pd.to_datetime(inputMktData['date'])
inputMktData['date']=inputMktData['date'].dt.strftime('%d-%m-%Y')
inputMktData['date'] =  pd.to_datetime(inputMktData['date'])
inputMktData['assetName'] = marketDataframe['assetName']
inputMktData['volume'] = marketDataframe['volume']
inputMktData['close'] = marketDataframe['close']
inputMktData['open'] = marketDataframe['open']
inputNewsData = pd.DataFrame()
inputNewsData['date'] = newsDataframe['time']
inputNewsData['date'] =  pd.to_datetime(inputNewsData['date'])
inputNewsData['date']=inputNewsData['date'].dt.strftime('%d-%m-%Y')
inputNewsData['date'] =  pd.to_datetime(inputNewsData['date'])
inputNewsData['headline'] = newsDataframe['headline']
inputNewsData['urgency'] = newsDataframe['urgency']
inputNewsData['subjects'] = newsDataframe['subjects']
inputNewsData['audiences'] = newsDataframe['audiences']
inputNewsData['audiences'] = newsDataframe['audiences']
inputNewsData['assetName'] = newsDataframe['assetName']
inputNewsData['relevance'] = newsDataframe['relevance']
inputNewsData['sentimentNegative'] = newsDataframe['sentimentNegative']
inputNewsData['sentimentNeutral'] = newsDataframe['sentimentNeutral']
inputNewsData['sentimentPositive'] = newsDataframe['sentimentPositive']
del newsDataframe,marketDataframe


CPU times: user 1min 39s, sys: 145 ms, total: 1min 40s
Wall time: 1min 40s


In [0]:
inputNewsData = inputNewsData.dropna()

In [8]:
inputNewsData.head()

,date,headline,urgency,subjects,audiences,assetName,relevance,sentimentNegative,sentimentNeutral,sentimentPositive
0,2007-01-01,China's Daqing pumps 43.41 mln tonnes of oil i...,3,"{'ENR', 'ASIA', 'CN', 'NGS', 'EMRG', 'RTRS', '...","{'Z', 'O', 'OIL'}",PetroChina Co Ltd,0.235702,0.500739,0.419327,0.079934
1,2007-01-01,"FEATURE-In kidnapping, finesse works best",3,"{'FEA', 'CA', 'LATAM', 'MX', 'INS', 'ASIA', 'I...","{'PGE', 'PCO', 'G', 'ESN', 'MD', 'PCU', 'DNP',...",Travelers Companies Inc,0.447214,0.600082,0.345853,0.054064
2,2007-01-01,PRESS DIGEST - Wall Street Journal - Jan 1,3,"{'RET', 'ENR', 'ID', 'BG', 'US', 'PRESS', 'IQ'...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",Wal-Mart Stores Inc,0.377964,0.450049,0.295671,0.254280
3,2007-01-01,PRESS DIGEST - New York Times - Jan 1,3,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",Google Inc,0.149071,0.752917,0.162715,0.084368
4,2007-01-01,PRESS DIGEST - New York Times - Jan 1,3,"{'FUND', 'FIN', 'CA', 'SFWR', 'INS', 'PUB', 'B...","{'T', 'DNP', 'PSC', 'U', 'D', 'M', 'RNP', 'PTD...",XM Satellite Radio Holdings Inc,0.149071,0.699274,0.209360,0.091367


In [9]:
inputMktData.head()

,date,assetName,volume,close,open
0,2007-01-02,Agilent Technologies Inc,2606900.0,32.19,32.17
1,2007-01-02,AirTran Holdings Inc,2051600.0,11.12,11.08
2,2007-01-02,Advance Auto Parts Inc,1164800.0,37.51,37.99
3,2007-01-02,Apple Inc,23747329.0,84.74,86.23
4,2007-01-02,ABB Ltd,1208600.0,18.02,18.01


In [0]:
newsHeadlineData = inputNewsData['headline'].values

In [11]:
%%time
tagged_data = [TaggedDocument(words=word_tokenize(str(_d).lower()), tags=[str(i)]) for i, _d in enumerate(newsHeadlineData)]
max_epochs = 5
vec_size = 25
alpha = 0.025
model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
model.build_vocab(tagged_data)
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

model.save("d2v.model")
del tagged_data
print("Model Saved")

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app


iteration 1
iteration 2
iteration 3
iteration 4
Model Saved
CPU times: user 22min 55s, sys: 6min 36s, total: 29min 32s
Wall time: 19min 2s


In [0]:
def longestVector(arr):
  k=-1
  maximum = -2
  for i in range(len(arr)):
    k = len(arr.item(i))
    if k>maximum:
      maximum = k
      index = i
  regex = r"[']([a-zA-z]*)[']"
  line = arr.item(index)
  matchObj = re.findall(regex, line, re.M|re.I)
  length = len(matchObj)
  return length
  

In [0]:
def setOfAllCategories(arr):
  allCats = set()
  regex = r"[']([a-zA-z]*)[']"
  for i in range(len(arr)):
    line = arr.item(i)
    matchObj = re.findall(regex, line, re.M|re.I)
    for j in matchObj:
      allCats.add(j)
  return allCats

In [0]:
def createDictVector(numCat,nors,vs,arr):
  vectorFrame = pd.DataFrame(np.zeros((nors,vs)))
  DictionaryOfCategory = dict(numCat)
  DictCat = {v: k for k, v in DictionaryOfCategory.items()}
  regex = r"[']([a-zA-z]*)[']"
  for i in range(len(arr)):
    line = arr.item(i)
    matchObj = re.findall(regex, line, re.M|re.I)
    k=0
    for j in matchObj:
      vectorFrame.at[i,k] = DictCat[j] 
      k = k+1
  df = vectorFrame
  df['features'] = df[df.columns.values].values.tolist()
  vectorNpArray = df['features'].values
  return vectorNpArray

In [0]:
def vectorization(obj):
  print('Calculating the vector size')
  vectorSize = longestVector(obj)
  print('Creating a list of all unique category values')
  categories = setOfAllCategories(obj)
  categories = list(categories)
  numberOfRows = len(obj)
  print('Creating Numbered List of Categories')
  numberedListOfCategories = enumerate(categories)
  print('Making an array of feature')
  featureVector = createDictVector(numberedListOfCategories,numberOfRows,vectorSize,obj)
  print('Finished')
  return featureVector

In [16]:
subjectArray = inputNewsData['subjects'].values
featureVectorSubject = vectorization(subjectArray)
del subjectArray
print(featureVectorSubject)

Calculating the vector size
Creating a list of all unique category values
Creating Numbered List of Categories
Making an array of feature
Finished
[list([69.0, 2.0, 322.0, 568.0, 210.0, 477.0, 392.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 list([458.0, 482.0, 136.0, 361.0, 376.0, 2.0, 277.0, 605.0, 469.0, 200.0, 164.0, 477.0, 552.0, 295.0, 392.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 list([389.0, 69.0, 117.0, 642.0, 200.0, 159.0, 491.0, 525.0, 568.0, 472.0, 296.0, 477.0, 392.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [17]:
audiencesArray = inputNewsData['audiences'].values
featureVectorAudience = vectorization(audiencesArray)
del audiencesArray
print(featureVectorAudience)

Calculating the vector size
Creating a list of all unique category values
Creating Numbered List of Categories
Making an array of feature
Finished
[list([46.0, 16.0, 105.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 list([57.0, 79.0, 34.0, 63.0, 81.0, 87.0, 68.0, 7.0, 75.0, 26.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 list([89.0, 68.0, 7.0, 75.0, 44.0, 20.0, 26.0, 18.0, 107.0, 3.0, 93.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 ...
 list([3.0, 75.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 list([3.0, 75.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 list([3.0, 7.0, 75.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])]


In [0]:
def string2VecParser(obj):
  tempVector2 = []
  for i in range(len(obj)):
      stringConv = re.findall( r'([\d]*[.][\d]*)[,]', str(obj[i]))
      temp = []
      for j in stringConv:
        k = float(j)
        temp.append(k)
      tempVector = np.array(temp)
      tempVector2.append(tempVector)
  finalVector = np.array(tempVector2)
  return finalVector
  

In [0]:
def queryEngine(marketIndex, newsDate):
  td = timedelta(days=1)
  upper = -45
  lower = 20
  p = inputMktData.loc[(inputMktData['date'] <= newsDate - upper*td) & (inputMktData['assetName'] == marketIndex) & (inputMktData['date'] >= newsDate - lower*td)]
  if p.empty:
    return pd.Series((None,None,None,None,None,None))
  labelFrameTemp = p[p['date'] >= newsDate]
  labelFrameTemp = labelFrameTemp.reset_index()
  if len(labelFrameTemp.index) < 5:
    return pd.Series((None,None,None,None,None,None))
  labelFrameTemp2 = pd.DataFrame()
  labelFrameTemp2['volume']=labelFrameTemp['volume'] 
  labelFrameTemp2['open']=labelFrameTemp['open']
  labelFrameTemp2['close']=labelFrameTemp['close']
  labelFrame = labelFrameTemp2.T
  labelFrame['labelTemp'] = labelFrame[labelFrame.columns.values].values.tolist()
  featureFrameTemp = p[p['date'] < newsDate]
  featureFrameTemp = featureFrameTemp.reset_index()
  featureFrameTemp2 = pd.DataFrame()
  featureFrameTemp2['volume']=featureFrameTemp['volume']
  featureFrameTemp2['open']=featureFrameTemp['open']
  featureFrameTemp2['close']=featureFrameTemp['close']
  featureFrame = featureFrameTemp2.T
  featureFrame['featureTemp'] = featureFrame[featureFrame.columns.values].values.tolist()
  label = labelFrame['labelTemp'].values
  feature = featureFrame['featureTemp'].values
  return pd.Series((label[0][:5],label[1][:5],label[2][:5],feature[0],feature[1],feature[2])) #label[volume open close] and feature[volume open close]

In [0]:
def assembler(obj):
  assembledVector = pd.DataFrame()
  assembledVector['Vector'] = obj[obj.columns.values].values.tolist()
  return assembledVector

In [0]:
def docvec(obj):
  obj = str(obj).lower()
  data = word_tokenize(obj)
  return model.infer_vector(data)

In [0]:
def queryForPandas(row):
  return queryEngine(str(row['assetName']), row['date'])

In [23]:
%%time
#news parsing
inputNewsData['subjectFeatures'] = featureVectorSubject.tolist() #subject to vector
print('Subject Vectorized...')
del featureVectorSubject
inputNewsData['audienceFeatures'] = featureVectorAudience.tolist() #audience to Vector
del featureVectorAudience
print('Audiences Vectorized...')
print('Headlines Vectorization Started')
inputNewsData['headlineVector'] = inputNewsData['headline'].apply(docvec) #headline to Vector
del model
print('Headlines Vectorized...')
print('Creating Cleaner DataFrame')
NewsDataframeFinal = pd.DataFrame()
NewsDataframeFinal['date'] = inputNewsData['date'] 
NewsDataframeFinal['urgency'] = inputNewsData['urgency']
NewsDataframeFinal['relevance'] = inputNewsData['relevance']
NewsDataframeFinal['assetName'] = inputNewsData['assetName']
NewsDataframeFinal['sentimentNegative'] = inputNewsData['sentimentNegative']
NewsDataframeFinal['sentimentNeutral'] = inputNewsData['sentimentNeutral']
NewsDataframeFinal['sentimentPositive'] = inputNewsData['sentimentPositive']
NewsDataframeFinal['subjectFeatures'] = inputNewsData['subjectFeatures']
NewsDataframeFinal['audienceFeatures'] = inputNewsData['audienceFeatures']
NewsDataframeFinal['headlineVector'] = inputNewsData['headlineVector']
del inputNewsData
print('DataFrame Creation Complete...')


Subject Vectorized...
Audiences Vectorized...
Headlines Vectorization Started
Headlines Vectorized...
Creating Cleaner DataFrame
DataFrame Creation Complete...
CPU times: user 3min 51s, sys: 0 ns, total: 3min 51s
Wall time: 3min 51s


In [24]:
%%time
print('Query Engine Started...')
NewsDataframeFinal['labelVolume'] = np.nan
NewsDataframeFinal['labelOpen'] = np.nan
NewsDataframeFinal['labelClose'] = np.nan
NewsDataframeFinal['featureVolume'] = np.nan
NewsDataframeFinal['featureOpen'] = np.nan
NewsDataframeFinal['featureClose'] = np.nan
NewsDataframeFinal[['labelVolume','labelOpen','labelClose','featureVolume','featureOpen','featureClose']] = NewsDataframeFinal[['labelVolume','labelOpen','labelClose','featureVolume','featureOpen','featureClose']].astype(object)
NewsDataframeFinal[['labelVolume','labelOpen','labelClose','featureVolume','featureOpen','featureClose']] = NewsDataframeFinal.apply(queryForPandas, axis = 1)
NewsDataframeFinal = NewsDataframeFinal[NewsDataframeFinal.notnull()]
print('Query Engine Task Finished...')

Query Engine Started...
Query Engine Task Finished...
CPU times: user 2h 45min 8s, sys: 0 ns, total: 2h 45min 8s
Wall time: 2h 45min 13s


In [25]:
NewsDataframeFinal.head()

,date,urgency,relevance,assetName,sentimentNegative,sentimentNeutral,sentimentPositive,subjectFeatures,audienceFeatures,headlineVector,labelVolume,labelOpen,labelClose,featureVolume,featureOpen,featureClose
0,2007-01-01,3,0.235702,PetroChina Co Ltd,0.500739,0.419327,0.079934,"[69.0, 2.0, 322.0, 568.0, 210.0, 477.0, 392.0,...","[46.0, 16.0, 105.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[-0.06354199, -0.03799604, -0.12417433, -0.012...","[838700.0, 687700.0, 628836.0, 665539.0, 40430...","[122.7, 124.44, 121.95, 121.95, 122.25]","[123.4, 124.15, 122.8, 122.88, 121.8]",[],[],[]
1,2007-01-01,3,0.447214,Travelers Companies Inc,0.600082,0.345853,0.054064,"[458.0, 482.0, 136.0, 361.0, 376.0, 2.0, 277.0...","[57.0, 79.0, 34.0, 63.0, 81.0, 87.0, 68.0, 7.0...","[0.09221249, -0.041277327, -0.033864953, -0.23...","[1513000.0, 2626171.0, 2128988.0, 2679145.0, 2...","[51.27, 52.97, 54.8, 54.17, 53.73]","[51.85, 54.44, 54.6, 54.58, 54.48]",[],[],[]
2,2007-01-01,3,0.377964,Wal-Mart Stores Inc,0.450049,0.295671,0.254280,"[389.0, 69.0, 117.0, 642.0, 200.0, 159.0, 491....","[89.0, 68.0, 7.0, 75.0, 44.0, 20.0, 26.0, 18.0...","[0.08623818, 0.060358655, 0.011216616, -0.0078...",NaN,NaN,NaN,NaN,NaN,NaN
3,2007-01-01,3,0.149071,Google Inc,0.752917,0.162715,0.084368,"[22.0, 486.0, 482.0, 538.0, 376.0, 369.0, 119....","[89.0, 68.0, 7.0, 75.0, 44.0, 20.0, 26.0, 18.0...","[0.0030525122, 0.0105378665, -0.0068828673, 0....",NaN,NaN,NaN,NaN,NaN,NaN
4,2007-01-01,3,0.149071,XM Satellite Radio Holdings Inc,0.699274,0.209360,0.091367,"[22.0, 486.0, 482.0, 538.0, 376.0, 369.0, 119....","[89.0, 68.0, 7.0, 75.0, 44.0, 20.0, 26.0, 18.0...","[-0.035201706, 0.051901802, -0.0036501635, -0....","[6452689.0, 5096016.0, 6724300.0, 7297105.0, 7...","[14.14, 14.15, 13.01, 13.18, 13.0]","[14.15, 14.21, 13.06, 13.22, 12.98]",[],[],[]


In [26]:
NewsDataframeFinal = NewsDataframeFinal[NewsDataframeFinal.notnull()]
NewsDataframeFinal = NewsDataframeFinal.dropna()
NewsDataframeFinal.head()

,date,urgency,relevance,assetName,sentimentNegative,sentimentNeutral,sentimentPositive,subjectFeatures,audienceFeatures,headlineVector,labelVolume,labelOpen,labelClose,featureVolume,featureOpen,featureClose
0,2007-01-01,3,0.235702,PetroChina Co Ltd,0.500739,0.419327,0.079934,"[69.0, 2.0, 322.0, 568.0, 210.0, 477.0, 392.0,...","[46.0, 16.0, 105.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[-0.06354199, -0.03799604, -0.12417433, -0.012...","[838700.0, 687700.0, 628836.0, 665539.0, 40430...","[122.7, 124.44, 121.95, 121.95, 122.25]","[123.4, 124.15, 122.8, 122.88, 121.8]",[],[],[]
1,2007-01-01,3,0.447214,Travelers Companies Inc,0.600082,0.345853,0.054064,"[458.0, 482.0, 136.0, 361.0, 376.0, 2.0, 277.0...","[57.0, 79.0, 34.0, 63.0, 81.0, 87.0, 68.0, 7.0...","[0.09221249, -0.041277327, -0.033864953, -0.23...","[1513000.0, 2626171.0, 2128988.0, 2679145.0, 2...","[51.27, 52.97, 54.8, 54.17, 53.73]","[51.85, 54.44, 54.6, 54.58, 54.48]",[],[],[]
4,2007-01-01,3,0.149071,XM Satellite Radio Holdings Inc,0.699274,0.209360,0.091367,"[22.0, 486.0, 482.0, 538.0, 376.0, 369.0, 119....","[89.0, 68.0, 7.0, 75.0, 44.0, 20.0, 26.0, 18.0...","[-0.035201706, 0.051901802, -0.0036501635, -0....","[6452689.0, 5096016.0, 6724300.0, 7297105.0, 7...","[14.14, 14.15, 13.01, 13.18, 13.0]","[14.15, 14.21, 13.06, 13.22, 12.98]",[],[],[]
6,2007-01-01,3,0.149071,Walt Disney Co,0.152658,0.615878,0.231464,"[22.0, 486.0, 482.0, 538.0, 376.0, 369.0, 119....","[89.0, 68.0, 7.0, 75.0, 44.0, 20.0, 26.0, 18.0...","[0.010929393, 0.01853853, 0.015210629, -0.0617...","[8048100.0, 7670300.0, 12848592.0, 11835125.0,...","[35.09, 35.02, 34.07, 34.58, 34.7]","[35.01, 35.18, 34.59, 34.75, 34.67]",[],[],[]
7,2007-01-01,3,0.447214,Microsoft Corp,0.325929,0.374173,0.299898,"[22.0, 486.0, 482.0, 538.0, 376.0, 369.0, 119....","[89.0, 68.0, 7.0, 75.0, 44.0, 20.0, 26.0, 18.0...","[4.1332096e-06, 0.017256232, 0.024086958, -0.0...","[55367062.0, 60444038.0, 51763813.0, 55591266....","[30.84, 30.825, 29.045, 29.165, 29.57]","[30.56, 30.19, 29.03, 29.4, 29.46]",[],[],[]


In [0]:
def flat_list(obj):  
  listVal = [y for x in obj for y in x]
  return listVal

In [0]:
Data = pd.DataFrame()
Data['label'] = NewsDataframeFinal[['labelVolume','labelOpen','labelClose']].values.tolist()
Data['sentimentFeature'] = NewsDataframeFinal[['urgency','relevance','sentimentPositive','sentimentNeutral','sentimentNegative']].values.tolist()
Data['newsFeature'] = NewsDataframeFinal[['subjectFeatures','audienceFeatures','headlineVector','featureVolume','featureOpen','featureClose']].values.tolist()
Data['newsFeature'] = Data['newsFeature'].apply(flat_list)
del NewsDataframeFinal
Data['features'] = Data[['sentimentFeature','newsFeature']].values.tolist()
Data = Data.drop(columns=['sentimentFeature','newsFeature'])
Data['label'] = Data['label'].apply(flat_list)
Data['features'] = Data['features'].apply(flat_list)

In [29]:
Data.head()

,label,features
0,"[838700.0, 687700.0, 628836.0, 665539.0, 40430...","[3.0, 0.235702, 0.0799341, 0.41932700000000006..."
1,"[1513000.0, 2626171.0, 2128988.0, 2679145.0, 2...","[3.0, 0.44721400000000006, 0.0540643, 0.345853..."
2,"[6452689.0, 5096016.0, 6724300.0, 7297105.0, 7...","[3.0, 0.149071, 0.0913665, 0.20936, 0.699274, ..."
3,"[8048100.0, 7670300.0, 12848592.0, 11835125.0,...","[3.0, 0.149071, 0.23146399999999998, 0.6158779..."
4,"[55367062.0, 60444038.0, 51763813.0, 55591266....","[3.0, 0.44721400000000006, 0.299898, 0.374173,..."


In [30]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Conv1D, MaxPooling1D, Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from sklearn.preprocessing import normalize

features = Data['features'].values
labels = Data['label'].values

Using TensorFlow backend.


In [33]:
#hyperparameters
pool_size = 15
lstm_output_size = 15
batch_size = 10000
epochs = 5
max_features = 10000
maxlen = 200
embedding_size = 160

print('Loading data...')
x_train, x_test, y_train, y_test = train_test_split(features,labels, test_size= 0.2, random_state=21)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen, padding='post')
x_test = sequence.pad_sequences(x_test, maxlen=maxlen, padding='post')
y_train = sequence.pad_sequences(x_train, maxlen=15, padding='post')
y_test = sequence.pad_sequences(x_test, maxlen=15, padding='post')
x_train = normalize(x_train)
y_train = normalize(y_train)
x_test = normalize(x_test)
y_test = normalize(y_test)

Loading data...


In [34]:
print('Building model...')
model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.5))
model.add(Conv1D (kernel_size = (5), filters = 40, activation='relu'))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dropout(0.5))
model.add(Dense(15)) #prediction for next five days so the label should be made such that it has to be 15 dimensional(volume open close 5 days)
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
model.save('drive/My Drive/stockNewsDataset/my_model.hdf5')
print('Test score:', score)
print('Test accuracy:', acc)


Building model...
Train...
Train on 181728 samples, validate on 45432 samples
Epoch 1/5
181728/181728 [==============================] - 22s 119us/step - loss: 0.6274 - acc: 0.7051 - val_loss: 0.5148 - val_acc: 0.9328
Epoch 2/5
181728/181728 [==============================] - 17s 92us/step - loss: 0.4526 - acc: 0.8271 - val_loss: 0.3162 - val_acc: 0.9328
Epoch 3/5
181728/181728 [==============================] - 17s 92us/step - loss: 0.3270 - acc: 0.8861 - val_loss: 0.2169 - val_acc: 0.9328
Epoch 4/5
181728/181728 [==============================] - 17s 92us/step - loss: 0.2555 - acc: 0.9224 - val_loss: 0.1632 - val_acc: 0.9995
Epoch 5/5
45432/45432 [==============================] - 1s 21us/step
Test score: 0.12665072754143536
Test accuracy: 0.999474684943152
